In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('Tweets.csv')
df_new=df.copy()
df.drop(columns=['text'],inplace=True)
df

,textID,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD,negative
2,088c60f138,bullying me,negative
3,9642c003ef,leave me alone,negative
4,358bd9e861,"Sons of ****,",negative
...,...,...,...
27476,4eac33d1c0,d lost,negative
27477,4f4c4fc327,", don`t force",negative
27478,f67aae2310,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,positive


In [2]:
import string
punc=string.punctuation
punc=punc.replace("*","")
punc

'!"#$%&\'()+,-./:;<=>?@[\\]^_`{|}~'

In [3]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
lst=stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YATIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from nltk.stem import PorterStemmer,WordNetLemmatizer
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\YATIN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
from nltk.tokenize import word_tokenize,sent_tokenize
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\YATIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [6]:
def preprocess_text(sentence):
    sentence = sentence.lower() # Convert to lowercase
    punc = string.punctuation  # Remove punctuation
    sentence = "".join([i for i in sentence if i not in punc])
    tokens = word_tokenize(sentence) # Tokenize
    stop_words = set(stopwords.words('english'))#stopwords
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, "v") for token in tokens]
     # Join tokens back into a string
    processed_sentence = " ".join(lemmatized_tokens)
    return processed_sentence

In [7]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\YATIN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [9]:
df['selected_text']=df['selected_text'].astype(str) 
df['selected_text']=df['selected_text'].apply(func=preprocess_text)

In [10]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
df['sentiment']=encoder.fit_transform(df['sentiment'])

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
vectorize=CountVectorizer()
x_vector=vectorize.fit_transform(df['selected_text'])

In [12]:
y=df['sentiment']

In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_vector,y,test_size=0.3,random_state=42)

In [14]:
from sklearn.naive_bayes import MultinomialNB
model=MultinomialNB()
model.fit(x_train,y_train)

MultinomialNB()

In [15]:
y_pred=model.predict(x_test)

In [16]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report,accuracy_score

In [17]:
models=[SVC(),MultinomialNB()]
for model in models:
  print(model)
  model.fit(x_train,y_train)
  y_pred=model.predict(x_test)
  print(f'Accuracy:{round(accuracy_score(y_test,y_pred)*100)}')
  print(classification_report(y_test,y_pred))
  print('------------------------------------------')

SVC()
Accuracy:81
              precision    recall  f1-score   support

           0       0.75      0.76      0.76      2338
           1       0.80      0.88      0.84      3371
           2       0.90      0.76      0.82      2536

    accuracy                           0.81      8245
   macro avg       0.81      0.80      0.81      8245
weighted avg       0.81      0.81      0.81      8245

------------------------------------------
MultinomialNB()
Accuracy:75
              precision    recall  f1-score   support

           0       0.82      0.61      0.70      2338
           1       0.71      0.83      0.77      3371
           2       0.78      0.79      0.78      2536

    accuracy                           0.75      8245
   macro avg       0.77      0.74      0.75      8245
weighted avg       0.76      0.75      0.75      8245

------------------------------------------


In [18]:
from sklearn.model_selection import GridSearchCV,cross_val_score
print(cross_val_score(MultinomialNB(),x_vector,y,cv=6))

[0.75376555 0.75327511 0.75764192 0.76069869 0.76943231 0.75393013]


In [19]:
def predict(sentence):
  var1=preprocess_text(sentence)
  vector=vectorize.transform([var1])
  pred=model.predict(vector)[0]
  if pred==0:
    print("Negative Tweet")
  elif pred==1:
    print('Neutral Tweet')
  else:
    print('Positive Tweet')

In [20]:
predict("Just had the best customer service experience with @CompanyX! Their team is so helpful and friendly #CustomerService #CompanyX")

Positive Tweet


In [21]:
predict("Ugh, just had the WORST experience with @CompanyX's customer service. They were rude and unhelpful #CustomerServiceFail #CompanyX")

Negative Tweet


In [22]:
predict("Just got my new @BrandY laptop and I'm still figuring it out. Anyone have any tips? #Laptop #BrandY")

Neutral Tweet


In [23]:
import pickle

class TweetSentimentPredictor:
    def __init__(self, model, vectorizer, encoder):
        self.model = model
        self.vectorizer = vectorizer
        self.encoder = encoder
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def preprocess_text(self, sentence):
        import string
        sentence = sentence.lower()
        punc = string.punctuation
        sentence = "".join([i for i in sentence if i not in punc])
        tokens = word_tokenize(sentence)
        tokens = [word for word in tokens if word not in self.stop_words]
        lemmatized_tokens = [self.lemmatizer.lemmatize(token, "v") for token in tokens]
        return " ".join(lemmatized_tokens)

    def predict(self, sentence):
        processed = self.preprocess_text(sentence)
        vector = self.vectorizer.transform([processed])
        pred = self.model.predict(vector)[0]
        label = self.encoder.inverse_transform([pred])[0]
        return f"Predicted Sentiment: {label}"


In [24]:
# Assuming model, vectorize, and encoder are already defined and trained
predictor = TweetSentimentPredictor(model, vectorize, encoder)

In [25]:
with open("tweet_sentiment_model.pkl", "wb") as f:
    pickle.dump(predictor, f)

print("✅ Model saved as tweet_sentiment_model.pkl")

✅ Model saved as tweet_sentiment_model.pkl


In [26]:
with open("tweet_sentiment_model.pkl", "rb") as f:
    loaded_model = pickle.load(f)

# Use it to predict
print(loaded_model.predict("I am very happy today!"))


Predicted Sentiment: positive
